#Theoretical model for a mixture of charged colloids, with counterions and salt and a polymer depletant

**General notes**:
  1. This model works with free energies that have been non-dimensionalized by the thermal energy and are divided by the total number of particles  


###Required imports

---

In [29]:
import os
import csv
import pandas as pd
from math import pi, exp, sqrt, log
from numpy.linalg import det
from numpy import array

###Function to make the hard sphere RDF dictionary
This reads in all of the hard sphere g(r) csv files in "\data\hs_structure\" of the working directory and places each (in order of increasing volumne fraction) into a pandas dataframe. There is also an "r" column that correpsonds to the radial distance and appears at the very end of the dataframe

**NOTE**: the RDF files must start at zero volume fraction and increase in one fixed increment (automatically detected when read in)

---

In [2]:
#read each rdf file at each volume fraction and store in a dictionary within a dictionary
#with volume fraction as the first key and r as the second key for very rapid read
def load_hs_rdf_data(rdf_directory):    
    rdf_files = os.listdir(rdf_directory) 
    
    rdf_file = rdf_files[0] #extract first filename
    eta = rdf_file[-11:-4] #extract the volume fraction from filename
    rdf_data = pd.read_csv(filepath_or_buffer = rdf_directory + rdf_file, header = None, names = ["r", float(eta)]) 
    
    for rdf_file in rdf_files[1:]: #loop over the various files in the rdf_data folder
        eta = rdf_file[-11:-4]
        rdf_data_2 = pd.read_csv(filepath_or_buffer = rdf_directory + rdf_file, header = None, names = ["r", float(eta)])[float(eta)]
        rdf_data = pd.concat([rdf_data, rdf_data_2], axis=1)   
    
    rdf_data = rdf_data.reindex_axis(sorted(rdf_data.columns), axis=1) #make sure columns go in ascending order of volume fraction
        
    return rdf_data

###Function to create an RDF at a specified volume fraction
This works by linearly interpolating between the two nearest stored volume fractions.

---

In [3]:
#actually creates the interpolation for the RDF
def interpolate(eta_step, eta_inc, gr):
    slope = (gr[1] - gr[0]) / eta_inc
    return (gr[0] + slope * eta_step)

#takes as input the desired volume fraction and the gr data in dataframe format
#and linearly interpolates between the two nearest stored volume fractions
def calculate_rdf(eta, d):
    rdf = pd.DataFrame()
    eta_inc = rdf_data.columns[1] #store the eta increment
    eta_max = rdf_data.columns[-2] #store the maximum eta to check if overstepped
    upper_index = int(eta / eta_inc) + 1 #for storing the nearest low and high volume fractions
    lower_index = int(eta / eta_inc)
    
    if eta >= eta_max:
        print "error: eta is too large to calculate!"
        return rdf
    try:
        eta_upper = rdf_data.columns[upper_index] #grab the lower and upper packing fractions for interpolation
        eta_lower = rdf_data.columns[lower_index]
    except:
        print "error: something went wrong with rdf calculation!"
        return rdf
    
    eta_step = eta - eta_lower #the step above the lower eta to take in the linear interpolation
    rdf["gr"] = rdf_data[[eta_lower, eta_upper]].apply(lambda x: interpolate(eta_step, eta_inc, x), axis = 1) 
    rdf["r"] = d*rdf_data["r"]  
    
    return rdf

###Calculate the effective volume fraction factor according to Hamad's theory to get effective RDF to be used for the two perturbative free energy terms

---

In [4]:
#evaluate the volume fraction factor for calculating the colloid-colloid AO RDF
def calculate_n_scale(components, x_c, x_p, #state variables  
                   d_c, d_p #system parameters
                  ):
    
    #set the needed parameters to compute the scale parameter
    b_2 = 4.0
    b_3 = 10.0
    Q = x_c*(d_c**3.0)
    
    #detect if for colloid-colloid or colloid-polymer
    #note: the x_c and x_p used should technically should sum to unity
    #but it does not matter due to the division step (see Santos paper for definitions)
    if components == "cc":
        C_c_cc = (b_3/b_2)*(d_c**3.0)
        C_p_cc = (d_p**3.0) + (b_3/b_2 - 1.0)*d_c*(d_p**2.0)
        w_cc = (b_2/b_3)*(x_c*C_c_cc + x_p*C_p_cc)/Q
        return w_cc
    elif components == "cp":
        C_c_cp = (d_c**3.0) + (b_3/b_2 - 1.0)*((2.0*d_p*(d_c**3.0))/(d_c + d_p))
        C_p_cp = 0.0
        w_cp = (b_2/b_3)*(x_c*C_c_cp + x_p*C_p_cp)/Q
        return w_cp
    
    #return a zero if no coded type used
    return 0

In [5]:
calculate_n_scale("cc", 0.01, 0.1, 1.0, 0.3)
#1.6480000000000001

1.6480000000000001

###Load RDF data into memory
The loaded RDF data will be used as a **global** variable to avoid having to copy things around from one function to another. All RDF's are normalized to use a core diameter of unit length which is then easily converted (by rescaling r) to another of different size when generating the desired interpolated RDF.

---

**Global structure variable**

In [6]:
rdf_data = load_hs_rdf_data("data/hs_structure/")

Example calculation of an RDF follows as:

In [7]:
rdf = calculate_rdf(0.45, 3.0)
if not rdf.empty:
    print rdf.head()

         gr      r
0  4.611920  3.000
1  4.458095  3.015
2  4.309468  3.030
3  4.167159  3.045
4  4.030865  3.060


In [8]:
rdf = calculate_rdf(0.61, 2.0)
if not rdf.empty:
    print rdf.head()

error: eta is too large to calculate!


###Calculates ideal free energy contribution
This is comprised of the large colloids (or polyions), the polymer and the two monovalent ions. The form I use below is technically incorrect as the density has dimensions and I apply a log to it. This is irrelevant though as any arbitrary constant volume multiplicative factor can be introduced in the log without penalty. Upon taking a derivative with repect to density, one would find this term cancels itself. This is the same as saying the free energy is totally arbitrary to within a constant additive factor

---

In [9]:
def calculate_f_id(p, x_c, x_p, x_pos, x_neg):
    
    #ideal contribution containing both mixing and translational entropy
    fid = x_c*log(x_c) + x_p*log(x_p) + x_pos*log(x_pos) + x_neg*log(x_neg) + log(p) - 1.0
    
    return fid

###Calculates perturbative polyion-polyion DLVO free energy contribution
I use an approximation due to Hamad which says the non-additive RDF contact values can be determined from a purely hard sphere contact value at a rescaled volume fraction. I go beyond this and extend this argument to say the whole RDF can be approximated as this is needed for the repulsive DLVO pair contribution to the free energy.

---

In [10]:
def cc_perturb_integral(x_c, x_p, #state variables
                        n_c, k, #redundant state variables already calculated in calling function
                        l, Z_c, d_c, d_p #system parameters
                        ):
    
    #effective charge parameter to be used in calculating the DLVO colloid-colloid pair interaction
    Z_eff = Z_c*(exp(k*(d_c/2.0))/(1.0 + k*(d_c/2.0)))
    
    #calculate the AO rdf according to Hamad with extension to the whole RDF (Hamad only used for contact value)
    w_cc = calculate_n_scale("cc", x_c, x_p, d_c, d_p)
    rdf = calculate_rdf(w_cc*n_c, d_c)
    
    #calculate the integral
    #the rdf only contains data outside the core region
    dr = rdf["r"][1] - rdf["r"][0]
    integral = 0.0
    for index in rdf.index.values[:-1]:
        r_1 = rdf["r"][index]
        r_2 = rdf["r"][index + 1]
        gr_1 = rdf["gr"][index]
        gr_2 = rdf["gr"][index + 1]
        Bur_1 = (Z_eff**2.0)*l*(exp(-k*r_1)/r_1)
        Bur_2 = (Z_eff**2.0)*l*(exp(-k*r_2)/r_2)
        fr_1 = exp(-Bur_1) - 1.0
        fr_2 = exp(-Bur_2) - 1.0
        integral = integral + (1.0/2.0)*dr*((r_1**2.0)*gr_1*fr_1 + (r_2**2.0)*gr_2*fr_2)
        
    return integral

#############################################################################################

def calculate_f_ex_dlvo(p, x_c, x_p, x_pos, x_neg, #state variables
                        l, Z_c, d_c, d_p #system parameters
                        ):
    
    #calculate various other parameters that make the below expression a little cleaner
    #ion variables
    p_pos = x_pos*p
    p_neg = x_neg*p
    p_ions = p_pos + p_neg
    s = (2.0*p_pos*p_neg)/p_ions
    k = sqrt(4.0*pi*l*p_ions)
    #colloid variables
    p_c = x_c*p
    n_c = (pi/6.0)*(d_c**3.0)*p_c
        
    #breaking up the various contributions for simplicity
    f_ex_dlvo_ion_coord = -(x_c*(Z_c**2.0)*(l/d_c))*((k*(d_c/2.0))/(1.0 + k*(d_c/2.0)))
    f_ex_dlvo_exc_vol = (1.0/p)*((n_c*s)/(1.0-n_c))
    f_ex_dlvo_neg_mean_field = -(1.0/p)*(1.0/2.0)*((4.0*pi*l*((p_c*Z_c)**2.0))/(k**2.0))
    f_ex_dlvo_pair_perturb = -2.0*pi*(x_c**2.0)*p*cc_perturb_integral(x_c, x_p, n_c, k, l, Z_c, d_c, d_p)
    
    #total DLVO contribution to the free energy
    f_ex_dlvo = f_ex_dlvo_ion_coord + f_ex_dlvo_exc_vol + f_ex_dlvo_neg_mean_field + f_ex_dlvo_pair_perturb
    
    return f_ex_dlvo

**Checking for scale invariance**

In [11]:
d_c = 1.0 
R_G = 0.3/2.0 
l = 0.1 
Z_c = 100.0

n_c = 0.025

p_c = 6.0*n_c/(pi*d_c*d_c*d_c)
p_p = 0.5*p_c
p_pos = 10.0*p_c
p_neg = 20.0*p_c

p = p_c + p_p + p_pos + p_neg
x_c = p_c/p
x_p = p_p/p
x_pos = p_pos/p
x_neg = p_neg/p

#calculate density scale factor
w_cc = calculate_n_scale_for_cc(x_c, x_p, d_c, R_G)
#calculate g(r) at contact
rdf = calculate_rdf(w_cc*n_c, d_c, rdf_data)
#calculate the free energy contribution
calculate_f_ex_dlvo(p, x_c, x_pos, x_neg, l, Z_c, d_c, rdf)

NameError: name 'calculate_n_scale_for_cc' is not defined

In [13]:
d_c = 1.0
d_p = 0.30
l = 0.1 
Z_c = 100.0

n_c = 0.025

p_c = 6.0*n_c/(pi*d_c*d_c*d_c)
p_p = 0.5*p_c
p_pos = 10.0*p_c
p_neg = 20.0*p_c

p = p_c + p_p + p_pos + p_neg
x_c = p_c/p
x_p = p_p/p
x_pos = p_pos/p
x_neg = p_neg/p

#calculate the free energy contribution
calculate_f_ex_dlvo(p, x_c, x_p, x_pos, x_neg, l, Z_c, d_c, d_p)
#-17.67148544525654

-17.67148544525654

###Calculates the perturbative polyion-polymer adsorption free energy contribution
This uses a perturbative inclusion of short range attractions between the polyions and polymers. These attractions can be throught of as including both VDW and dipole-ion interactions

---

In [14]:
def calculate_f_ex_int(p, x_c, x_p, #state variables
                        d_c, d_p, R_cp, BA, #system parameters
                        ):
    
    #transform to some helpful variables
    p_c = x_c*p
    n_c = (pi/6.0)*(d_c**3.0)*p_c
    R_G = d_p/2.0
    
    #calculate the AO rdf according to Hamad with extension to the whole RDF (Hamad only used for contact value)
    w_cp = calculate_n_scale("cp", x_c, x_p, d_c, d_p)
    g_contact = calculate_rdf(w_cp*n_c, (d_c/2.0 + R_G))["gr"][0]
    
    #interfacial free energy contribution
    f_ex_int = -4.0*pi*p*x_c*x_p*R_cp*((d_c/2.0 + R_G)**2.0)*g_contact*(exp(BA)-1.0)
    
    return f_ex_int

**Checking for scale invariance**

In [15]:
d_c = 1.0 #1.0
d_p = 0.30 #0.3
R_cp = 0.01 #0.05
BA = 13.0

n_c = 0.025

p_c = 6.0*n_c/(pi*d_c*d_c*d_c)
p_p = 0.5*p_c
p_pos = 10.0*p_c
p_neg = 20.0*p_c

p = p_c + p_p + p_pos + p_neg
x_c = p_c/p
x_p = p_p/p
x_pos = p_pos/p
x_neg = p_neg/p


calculate_f_ex_int(p, x_c, x_p, d_c, d_p, R_cp, BA)

#-18.577585661644907

-18.577585661644907

###Calculates the excess contribution of the non-addittive AO model to the free energy
This uses the approximate equation of state of Woodward et al. I use this as apposed to the Santos theory as it is simpler and yields similar results to the free volume form which agrees well with Santos.

---

In [16]:
def calculate_f_ex_ao(p, x_c, x_p, #state variables
                        d_c, d_p #system parameters
                      ):
    
    #calculate the needed densities
    R_G = d_p/2.0
    p_c = p*x_c
    p_p = p*x_p
    
    #the effective volume fraction for use in the carnahan starling free energy
    n_eff = (pi/6.0)*(p_c/(p_c + p_p))*(p_c*(d_c**3.0) + p_p*((d_c/2.0 + R_G)**3.0))
    
    #modified carnahan starling free energy
    f_ex_ao = (x_c + x_p)*(-3.0 + 2.0/(1.0 - n_eff) + 1.0/((1.0 - n_eff)**2.0))
    
    return f_ex_ao   

###Calculates the total free energy of the colloid-polymer-ion system
This just calls the four separate free energy functions and adds them all up. 

Up to this point I have treated the colloid and ions all as separate entities. Formally this is fine but electroneutrality ultimately sets a linear constraint on what combinations of the three charges species are possible. For this I first assume that the charge on the colloid is positive (as it is in the experimental system) which means the density of positive ions is the density of the added salt in this simple system. Thus, when performing phase coexistence calculations the phases can be expressed as the amount of colloid, polymer and extra salt.

---

In [24]:
def calculate_f_explicit_ions(p, x_c, x_p, x_pos, x_neg, #state variables
               d_c, d_p, #particle sizes
               l, Z_c, #ionic solution parameters
               R_cp, BA #interfacial attraction parameters
               ):
    
    #ideal
    f_id = calculate_f_id(p, x_c, x_p, x_pos, x_neg)
    #excess dlvo
    f_ex_dlvo = calculate_f_ex_dlvo(p, x_c, x_p, x_pos, x_neg, l, Z_c, d_c, d_p) 
    #excess c-p interfacial
    f_ex_int = calculate_f_ex_int(p, x_c, x_p, d_c, d_p, R_cp, BA)
    #excess ao
    f_ex_ao = calculate_f_ex_ao(p, x_c, x_p, d_c, d_p)
    
    f = f_id + f_ex_dlvo + f_ex_int + f_ex_ao
    
    return f

This is an alternative form that expresses the free energy in terms of densities alone by simply referencing the other form--this makes the stability calculation easier to work with.

In [25]:
def calculate_f(p_c, p_p, p_s, #state variables
               d_c, d_p, #particle sizes
               l, Z_c, #ionic solution parameters
               R_cp, BA #interfacial attraction parameters
               ):
    
    #calculate the various densities based on specifying the salt density and the colloid and polymer
    if Z_c > 0.0:
        p_pos = p_s
        p_neg = p_pos + Z_c*p_c #charge neutrality condition
    else:
        p_neg = p_s
        p_pos = p_neg - Z_c*p_c #charge neutrality condition
    
    #calculate the total density and the mole fractions
    p = p_c + p_p + p_pos + p_neg
    x_c = p_c/p
    x_p = p_p/p
    x_pos = p_pos/p
    x_neg = p_neg/p
    
    f = calculate_f_explicit_ions(p, x_c, x_p, x_pos, x_neg, d_c, d_p, l, Z_c, R_cp, BA)
    
    return f

###Function for calculating the stability matrix and its determinant
This calculates the determinant of the 3x3 stability matrix for use in finding homogenous phase spinodal limits. The matrix is only 3x3 despite the 4 components due to charge neutrality. The particular choice I make for the matrix is the 3x3 of chemical potential derivatives with respect to the colloid, polymer and salt particle numbers.

---

In [ ]:
def calculate_stability_determinant(p_c, p_p, p_s, #state variables
                                    d_c, d_p, #particle sizes
                                    l, Z_c, #ionic solution parameters 
                                    R_cp, BA #interfacial attraction parameters
                                    ):
    #full density
    p =     
    
    #calculate the various stability matrix elements
    #first order derivatives
    df_c = calculate_f_derivatives("c", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    df_p = calculate_f_derivatives("p", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    df_s = calculate_f_derivatives("s", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    #now the six double derivatives (3 same and 3 mixed)
        #same
    d2f_cc = calculate_f_derivatives("cc", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    d2f_pp = calculate_f_derivatives("pp", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    d2f_ss = calculate_f_derivatives("ss", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
        #mixed
    d2f_cp = calculate_f_derivatives("cp", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    d2f_cs = calculate_f_derivatives("cs", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    d2f_ps = calculate_f_derivatives("ps", p_c, p_p, p_s, d_c, d_p, l, Z_c, R_cp, BA)
    
    #now calculate the matrix elements
        #diagonal elements
    Z_cc = df_c + df_c + p*d2f_cc
    Z_pp = df_p + df_p + p*d2f_pp
    Z_ss = df_s + df_s + p*d2f_ss
        #off-diagonal elements (exploiting symmetry here)
    Z_cp = df_c + df_p + p*d2f_cp
    Z_pc = Z_cp
    Z_cs = df_c + df_s + p*d2f_cs
    Z_sc = Z_cs
    Z_ps = df_p + df_s + p*d2f_ps
    Z_sp = Z_ps
    
    #build the 3x3 stability matrix as a numpy array and calculate the determinant 
    stability_matrix = array([[Z_cc, Z_cp, Z_cs], [Z_pc, Z_pp, Z_ps], [Z_sc, Z_sp, Z_ss]])
    stability_determinant = det(stability_matrix)
    
    return stability_determinant

In [31]:
a = array([[1, 2, 3], [4, 5, 6], [4, 2, 6]])
print a
print det(a)

[[1 2 3]
 [4 5 6]
 [4 2 6]]
-18.0


**Sizes**

In [ ]:
d_c = 1.0 #diameter of colloid
d_p = 0.33 #diameter of the polymer (~2xR_G)

**Colloid packing fraction**

In [ ]:
n_c = 0.05 #set volume fraction of the colloids

**Number of polymers and ions per colloid**

In [ ]:


p_c = 6.0*n_c/(pi*d_c*d_c*d_c)
p_p = 0.5*p_c
p_pos = 10.0*p_c
p_neg = 20.0*p_c


p = 
x_c = 
x_p = 


In [10]:
i=3 

def test_func(x):
    x = 5
    return "hi"
    
print test_func(i)

print i

hi
3


In [ ]:
def calculate_f_ex_int(p, x_c, x_p, #state variables
                        d_c, R_G, R_cp, BA, #system parameters
                        g_AO #reference structure
                        ):
    
    #some convenient transformations of the various inputs
    #n_c = p*x_c*(pi/6.0)*(d_c**3.0)
    #n_p = p*x_p*(4.0/3.0)*pi*(R_G**3.0)
    #z_cp = ((4.0*R_G)/d_c)/(1.0 + (2.0*R_G)/d_c)
    
    #AO adjusted contact value
    #g_AO = (1.0/(1.0-n_c-n_p)) + (g_HS - 1.0/(1.0-n_c-n_p))*z_cp
    
    #interfacial free energy contribution
    f_ex_int = -4.0*pi*p*x_c*x_p*R_cp*((d_c/2.0 + R_G)**2.0)*g_AO*(exp(BA)-1.0)
    
    return f_ex_int

In [ ]:
def calculate_C(type_1, type_2, type_3)

def calculate_B_3(type_1, type_2, type_3, d_c, R_G):
    if type_1 == "c":
        d_1 = d_c


def calculate_f_ex_ao(p, x_c, x_p, #state variables
                        d_c, R_G #system parameters
                      ):
    
    #the dimensionless 2nd and 3rd hard sphere virial coefficients in 3D
    b_2 = 4.0
    b_3 = 10.0
    
    #various c parameters used to calculate the mixture virial coefficients
    #colloid terms
    C_c_cc = (b_3/b_2)*(d_c**3.0)
    C_c_pp = (d_c + 2.0*R_G)**3.0
    C_c_cp = (d_c**3.0) + (b_3/b_2 - 1.0)*((4.0*R_G*(d_c**3.0))/(d_c + 2.0*R_G))
    #polymer_terms
    C_p_cc = ((2.0*R_G)**3.0) + (b_3/b_2 - 1.0)*d_c*((2.0*R_G)**2.0)
    C_p_pp = 0.0
    C_p_cp = 0.0
    
    #calculate the various B parameters
    
    #some convenient transformations of the various inputs
    p_c = p*x_c
    p_p = p*x_p
    X_c = p_c/(p_c + p_p)
    X_p = p_p/(p_c + p_p)
    n_c = p_c*(pi/6.0)*(d_c**3.0)
    n_p = p_p*(4.0/3.0)*pi*(R_G**3.0)
    Q = X_c*(d_c**3.0)
    
    #calculate the reference hard sphere excess free energy
    f_ex_hs = calculate_f_ex_hs((n_c + n_p))
    
    #
    f_ex_ao_lower = log(1.0 - n_c - n_p)*((b_3*Q*B_2 - b_2*B_3)/((b_3 - b_2)*(Q**2.0)))
    f_ex_ao_upper = f_ex_hs*((B_3 - Q*B_2)/((b_3 - b_2)*(Q**2.0)))

In [ ]:


p_c = 6.0*n_c/(pi*d_c*d_c*d_c)
p_p = 0.5*p_c
p_pos = 10.0*p_c
p_neg = 20.0*p_c


p = 
x_c = 
x_p = 
